# CRUJRA  (no_CH4_no_spin_up)

## 1.Make the nml

In [ ]:
import itertools
import os
import subprocess
import glob, os, shutil, sys
import numpy as np
import pandas as pd
import xarray as xr
import time

def make_namelist(nml_input,nml_output,station_list,mode='no_spin_up',forcing='FLUXNET-CH4'):
    # open nml file and readlines
    with open(nml_input+f'US-Los_{forcing}_no_CH4.nml', 'r') as file:
        nml_content = file.readlines()

    # modify var
    for i, line in enumerate(nml_content):
        # replace CASE NAME
        if 'DEF_CASE_NAME' in line:
            nml_content[i] = f"DEF_CASE_NAME = '{station_list['SITE_ID']}'\n"
        if 'DEF_simulation_time%start_year' in line:
            nml_content[i] = f"DEF_simulation_time%start_year = {int(station_list['YEAR_START'])}\n"
        if 'DEF_simulation_time%end_year' in line:
            nml_content[i] = f"DEF_simulation_time%end_year = {int(station_list['YEAR_END'])}\n"
        if 'DEF_simulation_time%spinup_year' in line:
            nml_content[i] = f"DEF_simulation_time%spinup_year = {int(station_list['YEAR_START'])-1}\n"
        if 'DEF_simulation_time%spinup_month' in line:
            nml_content[i] = f"DEF_simulation_time%spinup_month = 1\n"
        if 'DEF_simulation_time%spinup_day' in line:
            nml_content[i] = f"DEF_simulation_time%spinup_day = 1\n"
        if 'DEF_simulation_time%spinup_sec' in line:
            nml_content[i] = f"DEF_simulation_time%spinup_sec = 0\n"
        if 'DEF_simulation_time%spinup_repeat' in line:
            nml_content[i] = f"DEF_simulation_time%spinup_repeat = 0\n"
        if 'SITE_fsitedata' in line:
            nml_content[i] = f"SITE_fsitedata = '{station_list['srfpath']}'\n"
        if 'DEF_dir_output' in line:
            nml_content[i] = f"DEF_dir_output = '/share/home/dq076/data/cases/site/{forcing}/{mode}/'\n"
        # if 'DEF_forcing_namelist' in line:
        #     nml_content[i] = f"DEF_forcing_namelist = '{nml_input}/cases/no_spin_up/forcing/SINGLE_{station_list['SITE_ID']}.nml'\n"

    # read back modified nml file
    # sometimes need modify nml file path

    new_file_path = f"{nml_output}{station_list['SITE_ID']}.nml"
    with open(new_file_path, 'w') as file:
        file.writelines(nml_content)

if __name__ == '__main__':
    mode='no_CH4_no_spin_up'
    forcing = 'CRUJRA'

    nml_input = "/share/home/dq076/mode/ME/CoLM202X_CH4_s/run/"
    nml_output = f"{nml_input}site/{forcing}/{mode}/"
    os.makedirs(nml_output, exist_ok=True)

    stnlist = f"/share/home/dq076/data/ME/FLUXNET-CH4/FLX_AA-Flx_CH4-META_20201112135337801132.csv"
    station_lists = pd.read_csv(stnlist, header=0)
    n = len(station_lists['SITE_ID'])
    for i in range(n):
        station_list = station_lists.iloc[i]
        station_list['srfpath'] = f'/share/home/dq076/data/CoLM_Forcing/PLUMBER2/Srfdata/{station_list['SITE_ID']}_{str(station_list['YEAR_START'])}-{str(station_list['YEAR_END'])}_FLUXNET-CH4_Srf.nc'     
        make_namelist(nml_input,nml_output,station_list,mode,forcing)

## 2.Run the CoLM

In [ ]:
import subprocess
import os
from joblib import Parallel, delayed
import glob
from concurrent.futures import ProcessPoolExecutor, as_completed  # 切换到 ProcessPoolExecutor 以支持实时进度

def load_environment(env_file):
    cmd = f'bash -c "source {env_file} && env"'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    new_env = {}
    for line in result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            if not key.startswith('BASH_FUNC_'):
                new_env[key] = value
    
    os.environ.update(new_env)
    return os.environ.copy()

def run_colm(run_path, nml_path, log_path, nml_name, updated_env):
    """
    处理单个 nml 文件，返回成功/失败状态（不打印进度，由主脚本处理）。
    """
    nml_file = f'{nml_path}{nml_name}.nml'
    log_file = f'{log_path}{nml_name}.txt'
    
    try:
        # 用 'w' 模式打开文件，重置内容
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== 处理 {nml_name}.nml ===\n")
            log.flush()
            
            commands = [
                [f'{run_path}mksrfdata.x', nml_file],
                [f'{run_path}mkinidata.x', nml_file],
                [f'{run_path}colm.x', nml_file]
            ]
            
            for cmd in commands:
                log.write(f"执行命令: {' '.join(cmd)}\n")
                log.flush()
                
                subprocess.run(cmd, 
                               env=updated_env, 
                               stdout=log, 
                               stderr=subprocess.STDOUT, 
                               text=True)
                
                log.write("\n" + "="*50 + "\n")
                log.flush()
            
            log.write(f"=== {nml_name} 处理完成 ===\n")
            log.flush()
        
        return True  # 成功
    except Exception as e:
        # 如果失败，也记录到日志
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== {nml_name}.nml 处理失败: {str(e)} ===\n")
        return False

if __name__ == "__main__":
    forcing ='CRUJRA'
    mode ='no_CH4_no_spin_up'

    env_file = '/share/home/dq089/soft/gnu-env'
    run_path = '/share/home/dq076/mode/ME/CoLM202X_CH4_s/run/'
    nml_path = f'{run_path}site/{forcing}/{mode}/'
    log_path = f'{nml_path}logs/'  
    os.makedirs(log_path, exist_ok=True)

    updated_env = load_environment(env_file)
    nml_files = glob.glob(f'{nml_path}*.nml')
    nml_names = [os.path.splitext(os.path.basename(nml_file))[0] for nml_file in nml_files]
    print(f"发现 {len(nml_files)} 个 .nml 文件：{nml_names}")
    
    # 切换到 ProcessPoolExecutor 以支持 as_completed 实时进度
    max_workers = min(24, os.cpu_count() or 1)
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务，返回 future 对象
        future_to_nml = {
            executor.submit(run_colm, run_path, nml_path, log_path, nml_name, updated_env): nml_name
            for nml_name in nml_names
        }
        
        # 维护剩余任务集合
        remaining_nml = set(nml_names)
        completed_count = 0
        
        # 实时监控完成
        for future in as_completed(future_to_nml):
            nml_name = future_to_nml[future]
            try:
                success = future.result()
                if success:
                    completed_count += 1
                remaining_nml.discard(nml_name)  # 移除已完成（无论成功/失败）
                
                # 打印进度：总数 + 剩余列表
                print(f"=== {nml_name} 处理完成（成功: {success}) ===")
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                else:
                    print("所有任务已完成！")
                print("-" * 50)
                
            except Exception as exc:
                print(f"{nml_name} 执行异常: {exc}")
                remaining_nml.discard(nml_name)
                completed_count += 1  # 视作完成（失败）
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                print("-" * 50)
    
    print("批量处理结束。")

## 3.Postprocess

In [ ]:
import glob
import os
from joblib import Parallel, delayed
import pandas as pd

def merge(i,station_lists,data_path):
    station_list = station_lists.iloc[i]
    case = station_list['SITE_ID']
    yrstt = station_list['YEAR_START']
    yrend = station_list['YEAR_END']
    history_path = f'{data_path}/{case}/history/'
    postdata_path = f'{data_path}/{case}/postdata/'
    postdata_name = f'{case}_hist_{yrstt}-{yrend}.nc'
    os.makedirs(postdata_path,exist_ok=True)

    nc_files = [f for f in os.listdir(history_path) if f.endswith('.nc')]
    if nc_files:
        if yrstt==yrend:
            os.system(f'cp {history_path}{case}_hist_{yrstt}.nc {postdata_path}{postdata_name}')
        else:
            os.system(f'cdo -O -mergetime {history_path}*.nc {postdata_path}{postdata_name}')

if __name__ == '__main__':
    forcing = 'CRUJRA'
    mode = 'no_CH4_no_spin_up'

    cases_path = '/share/home/dq076/data/cases/site/'
    data_path = f'{cases_path}/{forcing}/{mode}/'

    stnlist = f"/share/home/dq076/data/ME/FLUXNET-CH4/FLX_AA-Flx_CH4-META_20201112135337801132.csv"
    station_lists = pd.read_csv(stnlist, header=0)
    station_lists = station_lists[station_lists['FLUXNET-CH4_DATA_POLICY'] == 'CCBY4.0'].reset_index(drop=True)
    
    results = Parallel(n_jobs=24)(delayed(merge)(i,station_lists,data_path) for i in range(station_lists.shape[0]))